In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
# import libraries
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential 
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
# Helper libraries
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from google.colab import files
import io

In [3]:
# load mostly preprocessed data (preprocessing accomplished before 3/25 proposal)

# would use regular numpy syntax to load locally:
#X = np.load('<path>/X_cleaned_attempt_new.npy',allow_pickle=True)
#Y = np.load('<path>/Y_cleaned_attempt_new.npy',allow_pickle=True)
#X = np.asarray(X).astype('float32')
#Y = np.asarray(Y).astype('float32')

# i'm using colab, so it's slightly more complicated
ytrainval_byteseq = files.upload() # choose 'Y_cleaned_attempt_new.npy' from local system
ytrainval_filelike = io.BytesIO(ytrainval_byteseq['Y_cleaned_attempt_new.npy']) # create file-like object
ytrainval = np.load(ytrainval_filelike,allow_pickle=True) # create regular numpy array
xtrainval_byteseq = files.upload() # choose 'X_cleaned_attempt_new.npy' from local system
xtrainval_filelike = io.BytesIO(xtrainval_byteseq['X_cleaned_attempt_new.npy']) # create file-like object
xtrainval = np.load(xtrainval_filelike,allow_pickle=True) # create regular numpy array
ytest_byteseq = files.upload() # choose 'Y_test.npy' from local system
ytest_filelike = io.BytesIO(ytest_byteseq['Y_test.npy']) # create file-like object
ytest = np.load(ytest_filelike,allow_pickle=True) # create regular numpy array
xtest_byteseq = files.upload() # choose 'X_test.npy' from local system
xtest_filelike = io.BytesIO(xtest_byteseq['X_test.npy']) # create file-like object
xtest = np.load(xtest_filelike,allow_pickle=True) # create regular numpy array

Saving Y_cleaned_attempt_new.npy to Y_cleaned_attempt_new.npy


Saving X_cleaned_attempt_new.npy to X_cleaned_attempt_new.npy


Saving Y_test.npy to Y_test.npy


Saving X_test.npy to X_test.npy


In [4]:
#check shapes
print(xtrainval.shape)
print(ytrainval.shape)
print(xtest.shape)
print(ytest.shape)

#ytrainval needs to be converted to number representation of labels
feature_map = {'Rock':0,'Electronic':1,'Experimental':2,'Hip-Hop':3,'Folk':4,'Instrumental':5,'Pop':6,
              'International':7,'Classical':8,'Old-Time / Historic':9, 'Jazz':10,'Country':11,'Soul-RnB':12,
              'Spoken':13,'Blues':14,'Easy Listening':15}
for i in range(ytrainval.shape[0]):
    ytrainval[i] = float(feature_map[ytrainval[i]])
ytrainval[i] = np.asarray(ytrainval[i]).astype('float32')

print(ytrainval)
print(ytest)

(9349, 520)
(9349,)
(40249, 520)
(40249,)
[3.0 3.0 3.0 ... 4.0 0.0 array(0., dtype=float32)]
[3.0 6.0 0.0 ... 0.0 0.0 array(0., dtype=float32)]


In [5]:
# for the preliminary report, i did a roughly 80-20 split, but used the 20% as the training set to improve speed
# so will recombine and do a 90-10 split, using the 90% as training/validation
xfull = np.concatenate((xtrainval,xtest),axis=0)
yfull = np.concatenate((ytrainval,ytest),axis=0)
print(xfull.shape)
print(yfull.shape)

(49598, 520)
(49598,)


In [6]:
# finish preprocessing
'''
Look out for improbable event that not all classes included in validation set
resulting in incompatible shape (15 categories for one hot)
just run again if this happens
'''

# ~50000 instances in full set, so could get away with 90-10 split probably
xtrainval, xtest, ytrainval, ytest = train_test_split(xfull,yfull,train_size=int(0.9*(xtest.shape[0]+xtrainval.shape[0])))

# split into training and validation
xtrain, xval, ytrain, yval = train_test_split(xtrainval,ytrainval,train_size=int(0.9*xtrainval.shape[0]))

# one-hot encoding
ytrain_1hot = to_categorical(ytrain)
yval_1hot = to_categorical(yval)
ytest_1hot = to_categorical(ytest)

print(ytrain_1hot.shape)
print(yval_1hot.shape)
print(ytest_1hot.shape)

(40174, 16)
(4464, 16)
(4960, 16)


In [7]:
print(xtrain.shape)
print(xtrain[0,:])
print(xval[0,:])
print(xtest[0,:])
# starting with MLP; will need to un-flatten features into time-series signal for cnn implementation
# looks like data isn't normalized

(40174, 520)
[-3.09899390e-01  2.00813651e-01 -4.19419289e-01 -4.83177871e-01
 -4.88094240e-01 -5.28498113e-01  9.30651426e-01  3.37793541e+00
  2.23753214e+00  6.26284003e-01 -5.40368676e-01  1.61023021e-01
  6.80902779e-01  6.91087365e-01  6.85647011e-01  6.81586206e-01
  6.35318160e-01  6.81717515e-01  6.43619061e-01  6.64180756e-01
  6.64180756e-01  6.68109953e-01  6.91748261e-01  6.64101720e-01
  2.69591242e-01  2.71138757e-01  2.70915002e-01  2.66046971e-01
  2.59812683e-01  2.91525364e-01  2.03960612e-01  1.26284078e-01
  1.68915257e-01  2.64811546e-01  2.90303648e-01  2.76634455e-01
  2.68004894e-01  2.64780700e-01  2.53414512e-01  2.63145864e-01
  2.72416413e-01  3.08607847e-01  2.02602342e-01  8.82767066e-02
  1.51804999e-01  2.63520867e-01  2.75879860e-01  2.82813013e-01
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2.96847284

In [12]:
# this is a modification of what was previously here, which wasn't very successful
# further modifications are down at the bottom
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.3),
          layers.Dense(100,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=150, validation_data=(xval, yval_1hot))

Epoch 1/150
1256/1256 [==============================] - 48s 5ms/step - loss: 1.4474 - accuracy: 0.5604 - val_loss: 1.1439 - val_accuracy: 0.6371
Epoch 2/150
1256/1256 [==============================] - 6s 5ms/step - loss: 1.2389 - accuracy: 0.5994 - val_loss: 1.1164 - val_accuracy: 0.6470
Epoch 3/150
1256/1256 [==============================] - 6s 5ms/step - loss: 1.2062 - accuracy: 0.6074 - val_loss: 1.0895 - val_accuracy: 0.6541
Epoch 4/150
1256/1256 [==============================] - 6s 5ms/step - loss: 1.1839 - accuracy: 0.6149 - val_loss: 1.0678 - val_accuracy: 0.6599
Epoch 5/150
1256/1256 [==============================] - 7s 6ms/step - loss: 1.1572 - accuracy: 0.6228 - val_loss: 1.0494 - val_accuracy: 0.6606
Epoch 6/150
1256/1256 [==============================] - 7s 5ms/step - loss: 1.1445 - accuracy: 0.6276 - val_loss: 1.0456 - val_accuracy: 0.6653
Epoch 7/150
1256/1256 [==============================] - 6s 5ms/step - loss: 1.1317 - accuracy: 0.6312 - val_loss: 1.0386 - val_a

In [ ]:
# >98% accuracy on the training set is pretty good, suggesting sufficient model complexity
# but validation accuracy is stuck around 68%
# hopefully, adding regularization will improve this
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dense(512,kernel_initializer='lecun_normal',kernel_regularizer='l2',bias_regularizer='l2',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal',kernel_regularizer='l2',bias_regularizer='l2',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal',kernel_regularizer='l2',bias_regularizer='l2',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal',kernel_regularizer='l2',bias_regularizer='l2',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal',kernel_regularizer='l2',bias_regularizer='l2',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal',kernel_regularizer='l2',bias_regularizer='l2',),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=150, validation_data=(xval, yval_1hot))

Epoch 1/150
1256/1256 [==============================] - 13s 8ms/step - loss: 2.8038 - accuracy: 0.5412 - val_loss: 1.7316 - val_accuracy: 0.5421
Epoch 2/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.6599 - accuracy: 0.5577 - val_loss: 1.5745 - val_accuracy: 0.5878
Epoch 3/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.5971 - accuracy: 0.5654 - val_loss: 1.5197 - val_accuracy: 0.5912
Epoch 4/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.5391 - accuracy: 0.5757 - val_loss: 1.5722 - val_accuracy: 0.5591
Epoch 5/150
1256/1256 [==============================] - 11s 9ms/step - loss: 1.4767 - accuracy: 0.5882 - val_loss: 1.4549 - val_accuracy: 0.6013
Epoch 6/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.4359 - accuracy: 0.5985 - val_loss: 1.4082 - val_accuracy: 0.5981
Epoch 7/150
1256/1256 [==============================] - 11s 8ms/step - loss: 1.4080 - accuracy: 0.6004 - val_loss: 1.3716 -

In [ ]:
# that didn't work, will try dropout regularization
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(rate=0.2),
          layers.Dense(512,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(rate=0.2),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=150, validation_data=(xval, yval_1hot))

Epoch 1/150
1256/1256 [==============================] - 12s 8ms/step - loss: 1.4129 - accuracy: 0.5750 - val_loss: 1.1817 - val_accuracy: 0.6228
Epoch 2/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.2216 - accuracy: 0.6089 - val_loss: 1.1611 - val_accuracy: 0.6279
Epoch 3/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.1878 - accuracy: 0.6157 - val_loss: 1.1161 - val_accuracy: 0.6440
Epoch 4/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.1619 - accuracy: 0.6219 - val_loss: 1.1200 - val_accuracy: 0.6387
Epoch 5/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.1445 - accuracy: 0.6300 - val_loss: 1.0963 - val_accuracy: 0.6463
Epoch 6/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.1321 - accuracy: 0.6318 - val_loss: 1.0800 - val_accuracy: 0.6503
Epoch 7/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.1154 - accuracy: 0.6393 - val_loss: 1.1011 -

In [ ]:
# that didn't work, will try dropout regularization
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(rate=0.8),
          layers.Dense(512,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(rate=0.8),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(rate=0.8),
          layers.Dense(128,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=150, validation_data=(xval, yval_1hot))

Epoch 1/150
1256/1256 [==============================] - 12s 8ms/step - loss: 1.8519 - accuracy: 0.4137 - val_loss: 1.4552 - val_accuracy: 0.5426
Epoch 2/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.6124 - accuracy: 0.4661 - val_loss: 1.4001 - val_accuracy: 0.5562
Epoch 3/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.5813 - accuracy: 0.4786 - val_loss: 1.3630 - val_accuracy: 0.5699
Epoch 4/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.5632 - accuracy: 0.4827 - val_loss: 1.3476 - val_accuracy: 0.5726
Epoch 5/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.5504 - accuracy: 0.4865 - val_loss: 1.3405 - val_accuracy: 0.5744
Epoch 6/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.5410 - accuracy: 0.4884 - val_loss: 1.3364 - val_accuracy: 0.5744
Epoch 7/150
1256/1256 [==============================] - 11s 9ms/step - loss: 1.5372 - accuracy: 0.4920 - val_loss: 1.3222 -

KeyboardInterrupt: ignored

In [ ]:
# i suspect some irrelevant parameters, so trying l1 regulation
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dense(512,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.95)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=150, validation_data=(xval, yval_1hot))

Epoch 1/150
1256/1256 [==============================] - 13s 9ms/step - loss: 10.3641 - accuracy: 0.4803 - val_loss: 3.9701 - val_accuracy: 0.4733
Epoch 2/150
1256/1256 [==============================] - 11s 8ms/step - loss: 3.1762 - accuracy: 0.5197 - val_loss: 2.9228 - val_accuracy: 0.5233
Epoch 3/150
1256/1256 [==============================] - 11s 9ms/step - loss: 2.8422 - accuracy: 0.5354 - val_loss: 2.3132 - val_accuracy: 0.5441
Epoch 4/150
1256/1256 [==============================] - 11s 9ms/step - loss: 2.5269 - accuracy: 0.5466 - val_loss: 2.8277 - val_accuracy: 0.5488
Epoch 5/150
1256/1256 [==============================] - 11s 9ms/step - loss: 2.4113 - accuracy: 0.5474 - val_loss: 2.1340 - val_accuracy: 0.5656
Epoch 6/150
1256/1256 [==============================] - 11s 9ms/step - loss: 2.2687 - accuracy: 0.5548 - val_loss: 2.2618 - val_accuracy: 0.5677
Epoch 7/150
1256/1256 [==============================] - 11s 9ms/step - loss: 2.2004 - accuracy: 0.5566 - val_loss: 2.0772 

KeyboardInterrupt: ignored

In [ ]:
# i suspect some irrelevant parameters, so trying l1 regulation
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dense(512,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=150, validation_data=(xval, yval_1hot))

Epoch 1/150
1256/1256 [==============================] - 24s 8ms/step - loss: 10.4413 - accuracy: 0.4816 - val_loss: 3.5299 - val_accuracy: 0.5336
Epoch 2/150
1256/1256 [==============================] - 10s 8ms/step - loss: 3.2842 - accuracy: 0.5245 - val_loss: 2.5593 - val_accuracy: 0.5372
Epoch 3/150
1256/1256 [==============================] - 10s 8ms/step - loss: 2.9627 - accuracy: 0.5390 - val_loss: 2.8630 - val_accuracy: 0.5394
Epoch 4/150
1256/1256 [==============================] - 10s 8ms/step - loss: 2.6978 - accuracy: 0.5560 - val_loss: 2.5063 - val_accuracy: 0.5753
Epoch 5/150
1256/1256 [==============================] - 10s 8ms/step - loss: 2.5270 - accuracy: 0.5668 - val_loss: 2.4881 - val_accuracy: 0.5784
Epoch 6/150
1256/1256 [==============================] - 10s 8ms/step - loss: 2.3587 - accuracy: 0.5724 - val_loss: 2.4303 - val_accuracy: 0.5737
Epoch 7/150
1256/1256 [==============================] - 10s 8ms/step - loss: 2.2690 - accuracy: 0.5766 - val_loss: 2.2033 

KeyboardInterrupt: ignored

In [ ]:
# i suspect some irrelevant parameters, so trying l1 regulation
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dense(512,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.01,decay_steps=2000,decay_rate=0.3,)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=150, validation_data=(xval, yval_1hot))

Epoch 1/150
1256/1256 [==============================] - 12s 8ms/step - loss: 9.2636 - accuracy: 0.4375 - val_loss: 5.0132 - val_accuracy: 0.5067
Epoch 2/150
1256/1256 [==============================] - 11s 9ms/step - loss: 3.7299 - accuracy: 0.5231 - val_loss: 2.8536 - val_accuracy: 0.5314
Epoch 3/150
1256/1256 [==============================] - 10s 8ms/step - loss: 2.4462 - accuracy: 0.5423 - val_loss: 2.0749 - val_accuracy: 0.5551
Epoch 4/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.9433 - accuracy: 0.5513 - val_loss: 1.7787 - val_accuracy: 0.5659
Epoch 5/150
1256/1256 [==============================] - 11s 9ms/step - loss: 1.7155 - accuracy: 0.5599 - val_loss: 1.6260 - val_accuracy: 0.5753
Epoch 6/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.6071 - accuracy: 0.5635 - val_loss: 1.5425 - val_accuracy: 0.5836
Epoch 7/150
1256/1256 [==============================] - 10s 8ms/step - loss: 1.5553 - accuracy: 0.5656 - val_loss: 1.5057 -

KeyboardInterrupt: ignored

In [ ]:
# combining l1 with dropout
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(rate=0.4),
          layers.Dense(512,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(rate=0.4),
          layers.Dense(256,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1',),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.01,decay_steps=2000,decay_rate=0.3,)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=150, validation_data=(xval, yval_1hot))


Epoch 1/150
1256/1256 [==============================] - 13s 9ms/step - loss: 11.7728 - accuracy: 0.4032 - val_loss: 7.3489 - val_accuracy: 0.4364
Epoch 2/150
1256/1256 [==============================] - 11s 8ms/step - loss: 5.4752 - accuracy: 0.4459 - val_loss: 4.5335 - val_accuracy: 0.4895
Epoch 3/150
1256/1256 [==============================] - 11s 9ms/step - loss: 3.7206 - accuracy: 0.4703 - val_loss: 3.0647 - val_accuracy: 0.5213
Epoch 4/150
1256/1256 [==============================] - 11s 8ms/step - loss: 2.7128 - accuracy: 0.4852 - val_loss: 2.2973 - val_accuracy: 0.5231
Epoch 5/150
1256/1256 [==============================] - 11s 9ms/step - loss: 2.2062 - accuracy: 0.4950 - val_loss: 1.9500 - val_accuracy: 0.5417
Epoch 6/150
1256/1256 [==============================] - 11s 8ms/step - loss: 1.9372 - accuracy: 0.5014 - val_loss: 1.7380 - val_accuracy: 0.5565
Epoch 7/150
1256/1256 [==============================] - 11s 8ms/step - loss: 1.7935 - accuracy: 0.5115 - val_loss: 1.6381 

KeyboardInterrupt: ignored

In [ ]:
# combining l1 with dropout, adjusting learning rates to what appears to be optimal, removing bias regularization
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(rate=0.4),
          layers.Dense(512,kernel_initializer='lecun_normal',kernel_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(rate=0.4),
          layers.Dense(256,kernel_initializer='lecun_normal',kernel_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal',kernel_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal',kernel_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal',kernel_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal',kernel_regularizer='l1',),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99,)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=150, validation_data=(xval, yval_1hot))

Epoch 1/150
1256/1256 [==============================] - 13s 9ms/step - loss: 12.0282 - accuracy: 0.4343 - val_loss: 4.8906 - val_accuracy: 0.4467
Epoch 2/150
1256/1256 [==============================] - 11s 9ms/step - loss: 4.5223 - accuracy: 0.4605 - val_loss: 4.3225 - val_accuracy: 0.5081
Epoch 3/150
1256/1256 [==============================] - 10s 8ms/step - loss: 4.1898 - accuracy: 0.4710 - val_loss: 3.8579 - val_accuracy: 0.5177
Epoch 4/150
1256/1256 [==============================] - 11s 9ms/step - loss: 3.9195 - accuracy: 0.4764 - val_loss: 3.6148 - val_accuracy: 0.4857
Epoch 5/150
1256/1256 [==============================] - 12s 9ms/step - loss: 3.5627 - accuracy: 0.4817 - val_loss: 3.3943 - val_accuracy: 0.5181
Epoch 6/150
1256/1256 [==============================] - 11s 9ms/step - loss: 3.3063 - accuracy: 0.4790 - val_loss: 3.0029 - val_accuracy: 0.5202
Epoch 7/150
1256/1256 [==============================] - 11s 9ms/step - loss: 3.1808 - accuracy: 0.4878 - val_loss: 3.0086 

KeyboardInterrupt: ignored

In [ ]:
# combining l1 with dropout, adjusting learning rates to what appears to be optimal, removing bias regularization
# also shouldn't regularize final layer
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(rate=0.4),
          layers.Dense(512,kernel_initializer='lecun_normal',kernel_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(rate=0.4),
          layers.Dense(256,kernel_initializer='lecun_normal',kernel_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal',kernel_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal',kernel_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal',kernel_regularizer='l1',),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal',),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99,)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=150, validation_data=(xval, yval_1hot))

Epoch 1/150
1256/1256 [==============================] - 40s 9ms/step - loss: 11.9001 - accuracy: 0.4372 - val_loss: 5.1039 - val_accuracy: 0.4778
Epoch 2/150
1256/1256 [==============================] - 11s 9ms/step - loss: 4.8369 - accuracy: 0.4595 - val_loss: 4.6961 - val_accuracy: 0.4854
Epoch 3/150
1256/1256 [==============================] - 11s 9ms/step - loss: 4.3689 - accuracy: 0.4750 - val_loss: 4.1814 - val_accuracy: 0.5284
Epoch 4/150
1256/1256 [==============================] - 11s 9ms/step - loss: 4.0096 - accuracy: 0.4899 - val_loss: 3.7745 - val_accuracy: 0.5352
Epoch 5/150
1256/1256 [==============================] - 11s 9ms/step - loss: 3.6160 - accuracy: 0.5002 - val_loss: 3.5151 - val_accuracy: 0.5421
Epoch 6/150
1256/1256 [==============================] - 11s 9ms/step - loss: 3.3717 - accuracy: 0.4998 - val_loss: 3.1828 - val_accuracy: 0.5087
Epoch 7/150
1256/1256 [==============================] - 11s 9ms/step - loss: 3.2052 - accuracy: 0.5077 - val_loss: 3.0242 

In [ ]:
# let's see how these compare to random guessing accuracy
yval_sum = np.sum(yval_1hot, axis=0)
print(yval_sum)
print(yval_1hot.shape)
print(yval_sum.shape)

[1282.  841.  985.  332.  250.  154.  215.  126.  115.   37.   38.   15.
   15.   40.   17.    2.]
(4464, 16)
(16,)


In [14]:
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.3),
          layers.Dense(100,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          #layers.Dense(128,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=50, validation_data=(xval, yval_1hot))

Epoch 1/50
1256/1256 [==============================] - 8s 5ms/step - loss: 1.4333 - accuracy: 0.5647 - val_loss: 1.1566 - val_accuracy: 0.6290
Epoch 2/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.2349 - accuracy: 0.6016 - val_loss: 1.1059 - val_accuracy: 0.6465
Epoch 3/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.2012 - accuracy: 0.6104 - val_loss: 1.0938 - val_accuracy: 0.6456
Epoch 4/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.1739 - accuracy: 0.6188 - val_loss: 1.0896 - val_accuracy: 0.6514
Epoch 5/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.1577 - accuracy: 0.6235 - val_loss: 1.0543 - val_accuracy: 0.6647
Epoch 6/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.1408 - accuracy: 0.6283 - val_loss: 1.0261 - val_accuracy: 0.6647
Epoch 7/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.1259 - accuracy: 0.6313 - val_loss: 1.0258 - val_accuracy:

In [16]:
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.3),
          layers.Dense(200,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=50, validation_data=(xval, yval_1hot))

Epoch 1/50
1256/1256 [==============================] - 9s 6ms/step - loss: 1.4246 - accuracy: 0.5682 - val_loss: 1.1508 - val_accuracy: 0.6306
Epoch 2/50
1256/1256 [==============================] - 7s 5ms/step - loss: 1.2405 - accuracy: 0.6010 - val_loss: 1.1104 - val_accuracy: 0.6411
Epoch 3/50
1256/1256 [==============================] - 7s 6ms/step - loss: 1.2076 - accuracy: 0.6088 - val_loss: 1.0938 - val_accuracy: 0.6434
Epoch 4/50
1256/1256 [==============================] - 7s 6ms/step - loss: 1.1844 - accuracy: 0.6150 - val_loss: 1.0712 - val_accuracy: 0.6570
Epoch 5/50
1256/1256 [==============================] - 7s 5ms/step - loss: 1.1600 - accuracy: 0.6241 - val_loss: 1.0566 - val_accuracy: 0.6640
Epoch 6/50
1256/1256 [==============================] - 7s 6ms/step - loss: 1.1398 - accuracy: 0.6257 - val_loss: 1.0553 - val_accuracy: 0.6622
Epoch 7/50
1256/1256 [==============================] - 7s 6ms/step - loss: 1.1215 - accuracy: 0.6341 - val_loss: 1.0427 - val_accuracy:

In [8]:
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.3),
          layers.Dense(200,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          #layers.Dense(128,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          #layers.Dense(64,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=50, validation_data=(xval, yval_1hot))

Epoch 1/50
1256/1256 [==============================] - 13s 7ms/step - loss: 1.4180 - accuracy: 0.5701 - val_loss: 1.1520 - val_accuracy: 0.6279
Epoch 2/50
1256/1256 [==============================] - 8s 7ms/step - loss: 1.2274 - accuracy: 0.6042 - val_loss: 1.1130 - val_accuracy: 0.6443
Epoch 3/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.1920 - accuracy: 0.6127 - val_loss: 1.0948 - val_accuracy: 0.6478
Epoch 4/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.1682 - accuracy: 0.6206 - val_loss: 1.0765 - val_accuracy: 0.6499
Epoch 5/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.1488 - accuracy: 0.6229 - val_loss: 1.0821 - val_accuracy: 0.6447
Epoch 6/50
1256/1256 [==============================] - 8s 7ms/step - loss: 1.1323 - accuracy: 0.6301 - val_loss: 1.0407 - val_accuracy: 0.6599
Epoch 7/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.1137 - accuracy: 0.6354 - val_loss: 1.0301 - val_accuracy

In [9]:
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.3),
          layers.Dense(300,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          #layers.Dense(128,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          #layers.Dense(64,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=50, validation_data=(xval, yval_1hot))

Epoch 1/50
1256/1256 [==============================] - 10s 7ms/step - loss: 1.4079 - accuracy: 0.5711 - val_loss: 1.1681 - val_accuracy: 0.6272
Epoch 2/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.2300 - accuracy: 0.6025 - val_loss: 1.1189 - val_accuracy: 0.6431
Epoch 3/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.1903 - accuracy: 0.6151 - val_loss: 1.0771 - val_accuracy: 0.6526
Epoch 4/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.1714 - accuracy: 0.6209 - val_loss: 1.0693 - val_accuracy: 0.6552
Epoch 5/50
1256/1256 [==============================] - 10s 8ms/step - loss: 1.1490 - accuracy: 0.6281 - val_loss: 1.0596 - val_accuracy: 0.6564
Epoch 6/50
1256/1256 [==============================] - 8s 7ms/step - loss: 1.1248 - accuracy: 0.6343 - val_loss: 1.0339 - val_accuracy: 0.6624
Epoch 7/50
1256/1256 [==============================] - 8s 7ms/step - loss: 1.1117 - accuracy: 0.6366 - val_loss: 1.0361 - val_accurac

In [10]:
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(300,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('relu'),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('relu'),
          #layers.Dense(128,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          #layers.Dense(64,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=50, validation_data=(xval, yval_1hot))

Epoch 1/50
1256/1256 [==============================] - 10s 7ms/step - loss: 1.3996 - accuracy: 0.5724 - val_loss: 1.1249 - val_accuracy: 0.6384
Epoch 2/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.2012 - accuracy: 0.6109 - val_loss: 1.0698 - val_accuracy: 0.6577
Epoch 3/50
1256/1256 [==============================] - 8s 7ms/step - loss: 1.1562 - accuracy: 0.6224 - val_loss: 1.0455 - val_accuracy: 0.6591
Epoch 4/50
1256/1256 [==============================] - 8s 7ms/step - loss: 1.1239 - accuracy: 0.6310 - val_loss: 1.0219 - val_accuracy: 0.6725
Epoch 5/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.0954 - accuracy: 0.6365 - val_loss: 1.0181 - val_accuracy: 0.6756
Epoch 6/50
1256/1256 [==============================] - 8s 7ms/step - loss: 1.0729 - accuracy: 0.6469 - val_loss: 0.9943 - val_accuracy: 0.6799
Epoch 7/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.0469 - accuracy: 0.6530 - val_loss: 0.9719 - val_accuracy

In [11]:
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(150,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('relu'),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('relu'),
          #layers.Dense(128,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          #layers.Dense(64,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=50, validation_data=(xval, yval_1hot))

Epoch 1/50
1256/1256 [==============================] - 10s 7ms/step - loss: 1.4199 - accuracy: 0.5654 - val_loss: 1.1292 - val_accuracy: 0.6358
Epoch 2/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.2160 - accuracy: 0.6045 - val_loss: 1.0854 - val_accuracy: 0.6550
Epoch 3/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.1690 - accuracy: 0.6188 - val_loss: 1.0432 - val_accuracy: 0.6671
Epoch 4/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.1409 - accuracy: 0.6254 - val_loss: 1.0349 - val_accuracy: 0.6673
Epoch 5/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.1158 - accuracy: 0.6329 - val_loss: 1.0340 - val_accuracy: 0.6642
Epoch 6/50
1256/1256 [==============================] - 8s 7ms/step - loss: 1.0933 - accuracy: 0.6415 - val_loss: 0.9981 - val_accuracy: 0.6723
Epoch 7/50
1256/1256 [==============================] - 8s 7ms/step - loss: 1.0765 - accuracy: 0.6436 - val_loss: 0.9869 - val_accuracy

In [12]:
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(100,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('relu'),
          #layers.Dense(256,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('relu'),
          layers.Dense(128,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          #layers.Dense(64,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=50, validation_data=(xval, yval_1hot))

Epoch 1/50
1256/1256 [==============================] - 10s 7ms/step - loss: 1.4371 - accuracy: 0.5599 - val_loss: 1.1534 - val_accuracy: 0.6290
Epoch 2/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.2274 - accuracy: 0.6033 - val_loss: 1.1103 - val_accuracy: 0.6458
Epoch 3/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.1876 - accuracy: 0.6125 - val_loss: 1.0786 - val_accuracy: 0.6521
Epoch 4/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.1611 - accuracy: 0.6229 - val_loss: 1.0680 - val_accuracy: 0.6568
Epoch 5/50
1256/1256 [==============================] - 8s 7ms/step - loss: 1.1452 - accuracy: 0.6255 - val_loss: 1.0554 - val_accuracy: 0.6635
Epoch 6/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.1263 - accuracy: 0.6281 - val_loss: 1.0429 - val_accuracy: 0.6658
Epoch 7/50
1256/1256 [==============================] - 9s 7ms/step - loss: 1.1138 - accuracy: 0.6352 - val_loss: 1.0233 - val_accuracy

KeyboardInterrupt: ignored

In [13]:
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(100,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('relu'),
          #layers.Dense(256,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('relu'),
          #layers.Dense(128,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          #layers.Dense(64,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('relu'),
          layers.Dense(16,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=50, validation_data=(xval, yval_1hot))

Epoch 1/50
1256/1256 [==============================] - 12s 8ms/step - loss: 1.5505 - accuracy: 0.5358 - val_loss: 1.1927 - val_accuracy: 0.6290
Epoch 2/50
1256/1256 [==============================] - 10s 8ms/step - loss: 1.2757 - accuracy: 0.5922 - val_loss: 1.1415 - val_accuracy: 0.6358
Epoch 3/50
1256/1256 [==============================] - 10s 8ms/step - loss: 1.2342 - accuracy: 0.6012 - val_loss: 1.1039 - val_accuracy: 0.6445
Epoch 4/50
1256/1256 [==============================] - 10s 8ms/step - loss: 1.2094 - accuracy: 0.6119 - val_loss: 1.0986 - val_accuracy: 0.6503
Epoch 5/50
1256/1256 [==============================] - 10s 8ms/step - loss: 1.1924 - accuracy: 0.6157 - val_loss: 1.0802 - val_accuracy: 0.6470
Epoch 6/50
1256/1256 [==============================] - 10s 8ms/step - loss: 1.1704 - accuracy: 0.6211 - val_loss: 1.0643 - val_accuracy: 0.6591
Epoch 7/50
1256/1256 [==============================] - 10s 8ms/step - loss: 1.1614 - accuracy: 0.6219 - val_loss: 1.0538 - val_ac

In [14]:
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(50,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('relu'),
          #layers.Dense(256,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('relu'),
          #layers.Dense(128,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          #layers.Dense(64,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          layers.Dense(512,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('relu'),
          layers.Dense(16,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=50, validation_data=(xval, yval_1hot))

Epoch 1/50
1256/1256 [==============================] - 12s 8ms/step - loss: 1.4431 - accuracy: 0.5727 - val_loss: 1.1592 - val_accuracy: 0.6366
Epoch 2/50
1256/1256 [==============================] - 10s 8ms/step - loss: 1.1718 - accuracy: 0.6291 - val_loss: 1.0923 - val_accuracy: 0.6584
Epoch 3/50
1256/1256 [==============================] - 10s 8ms/step - loss: 1.1198 - accuracy: 0.6441 - val_loss: 1.0618 - val_accuracy: 0.6635
Epoch 4/50
1256/1256 [==============================] - 10s 8ms/step - loss: 1.0738 - accuracy: 0.6558 - val_loss: 1.0570 - val_accuracy: 0.6624
Epoch 5/50
1256/1256 [==============================] - 10s 8ms/step - loss: 1.0485 - accuracy: 0.6676 - val_loss: 1.0386 - val_accuracy: 0.6653
Epoch 6/50
1256/1256 [==============================] - 10s 8ms/step - loss: 1.0307 - accuracy: 0.6685 - val_loss: 1.0456 - val_accuracy: 0.6617
Epoch 7/50
1256/1256 [==============================] - 10s 8ms/step - loss: 1.0062 - accuracy: 0.6756 - val_loss: 1.0231 - val_ac

In [15]:
# this was one of the best performing models from earlier, i'll just let it go
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.3),
          layers.Dense(200,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          #layers.Dense(128,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          #layers.Dense(64,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=500, validation_data=(xval, yval_1hot))

Epoch 1/500
1256/1256 [==============================] - 10s 7ms/step - loss: 1.4226 - accuracy: 0.5664 - val_loss: 1.1653 - val_accuracy: 0.6288
Epoch 2/500
1256/1256 [==============================] - 9s 7ms/step - loss: 1.2267 - accuracy: 0.6034 - val_loss: 1.1129 - val_accuracy: 0.6546
Epoch 3/500
1256/1256 [==============================] - 8s 7ms/step - loss: 1.1940 - accuracy: 0.6137 - val_loss: 1.0987 - val_accuracy: 0.6478
Epoch 4/500
1256/1256 [==============================] - 8s 7ms/step - loss: 1.1734 - accuracy: 0.6170 - val_loss: 1.0802 - val_accuracy: 0.6508
Epoch 5/500
1256/1256 [==============================] - 8s 7ms/step - loss: 1.1497 - accuracy: 0.6248 - val_loss: 1.0732 - val_accuracy: 0.6546
Epoch 6/500
1256/1256 [==============================] - 9s 7ms/step - loss: 1.1307 - accuracy: 0.6329 - val_loss: 1.0554 - val_accuracy: 0.6559
Epoch 7/500
1256/1256 [==============================] - 9s 7ms/step - loss: 1.1108 - accuracy: 0.6350 - val_loss: 1.0343 - val_a

KeyboardInterrupt: ignored

In [ ]:
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(100,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('relu'),
          #layers.Dense(256,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('relu'),
          #layers.Dense(128,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          #layers.Dense(64,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('relu'),
          layers.Dense(16,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=500, validation_data=(xval, yval_1hot))

Epoch 1/500
1256/1256 [==============================] - 12s 8ms/step - loss: 1.5580 - accuracy: 0.5327 - val_loss: 1.2057 - val_accuracy: 0.6203
Epoch 2/500
1256/1256 [==============================] - 10s 8ms/step - loss: 1.2761 - accuracy: 0.5926 - val_loss: 1.1402 - val_accuracy: 0.6393
Epoch 3/500
1256/1256 [==============================] - 10s 8ms/step - loss: 1.2296 - accuracy: 0.6049 - val_loss: 1.1031 - val_accuracy: 0.6532
Epoch 4/500
1256/1256 [==============================] - 10s 8ms/step - loss: 1.2054 - accuracy: 0.6101 - val_loss: 1.0803 - val_accuracy: 0.6552
Epoch 5/500
1256/1256 [==============================] - 10s 8ms/step - loss: 1.1843 - accuracy: 0.6168 - val_loss: 1.0586 - val_accuracy: 0.6635
Epoch 6/500
1256/1256 [==============================] - 10s 8ms/step - loss: 1.1651 - accuracy: 0.6258 - val_loss: 1.0686 - val_accuracy: 0.6640
Epoch 7/500
1256/1256 [==============================] - 10s 8ms/step - loss: 1.1501 - accuracy: 0.6277 - val_loss: 1.0450 -